In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

In [2]:
df = pd.read_csv('alpha24_step15Stud.csv')
df.head()

,czas(min),alpha,temp,Ts
0,0,1,"6,50",2
1,15,0,"6,35","2,18"
2,30,0,"6,20","2,19"
3,45,1,"6,05","1,68"
4,60,1,"5,90","1,5"


In [3]:
df['temp'] = df['temp'].apply(lambda x: x.replace(',','.')).astype('float64')
df['Ts'] = df['Ts'].apply(lambda x: x.replace(',','.')).astype('float64')

In [4]:
df.head()

,czas(min),alpha,temp,Ts
0,0,1,6.50,2.00
1,15,0,6.35,2.18
2,30,0,6.20,2.19
3,45,1,6.05,1.68
4,60,1,5.90,1.50


In [5]:
df.rename(columns={"czas(min)": "time", "alpha": "compressor", "temp": 'ambient_temp', "Ts": 'refrigerator_temp'}, inplace=True)
df.set_index('time', inplace=True)
df.head()

,compressor,ambient_temp,refrigerator_temp
time,,,
0,1,6.50,2.00
15,0,6.35,2.18
30,0,6.20,2.19
45,1,6.05,1.68
60,1,5.90,1.50


In [6]:
size = 12
delay = 12
a = df.iloc[0:size,:].to_numpy().reshape(delay,size*3//delay)
a

array([[1.  , 6.5 , 2.  ],
       [0.  , 6.35, 2.18],
       [0.  , 6.2 , 2.19],
       [1.  , 6.05, 1.68],
       [1.  , 5.9 , 1.5 ],
       [0.  , 5.74, 1.64],
       [0.  , 5.59, 1.62],
       [0.  , 5.44, 1.56],
       [0.  , 5.3 , 1.54],
       [0.  , 5.26, 1.82],
       [0.  , 5.23, 1.86],
       [0.  , 5.21, 1.89]])

In [7]:
for predictor in range(a.shape[1] - 1):
    sorted_predictor = a[:,predictor].argsort()
    b = a[sorted_predictor]
    print(predictor, b, "\n")


0 [[0.   6.35 2.18]
 [0.   6.2  2.19]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]
 [1.   6.5  2.  ]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]] 

1 [[0.   5.21 1.89]
 [0.   5.23 1.86]
 [0.   5.26 1.82]
 [0.   5.3  1.54]
 [0.   5.44 1.56]
 [0.   5.59 1.62]
 [0.   5.74 1.64]
 [1.   5.9  1.5 ]
 [1.   6.05 1.68]
 [0.   6.2  2.19]
 [0.   6.35 2.18]
 [1.   6.5  2.  ]] 



In [8]:
SSR = lambda a: ((a.mean() - a)**2).sum()

for i in range(1,b.shape[0]):
    ssr_1 = SSR(b[:i, -1])
    ssr_2 = SSR(b[i:, -1])
    print(f"array: {b[:i, -1]} ssr = {ssr_1} \n\
             array {b[i:, -1]} ssr = {ssr_2} \n\
             SSR={ssr_1+ssr_2}")

array: [1.89] ssr = 0.0 
             array [1.86 1.82 1.54 1.56 1.62 1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.6240909090909093 
             SSR=0.6240909090909093
array: [1.89 1.86] ssr = 0.00044999999999999413 
             array [1.82 1.54 1.56 1.62 1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.61721 
             SSR=0.61766
array: [1.89 1.86 1.82] ssr = 0.002466666666666656 
             array [1.54 1.56 1.62 1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.6147555555555556 
             SSR=0.6172222222222222
array: [1.89 1.86 1.82 1.54] ssr = 0.07767499999999998 
             array [1.56 1.62 1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.5563875 
             SSR=0.6340625
array: [1.89 1.86 1.82 1.54 1.56] ssr = 0.11551999999999996 
             array [1.62 1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.4926000000000001 
             SSR=0.60812
array: [1.89 1.86 1.82 1.54 1.56 1.62] ssr = 0.12634999999999996 
             array [1.64 1.5  1.68 2.19 2.18 2.  ] ssr = 0.4411500000000001 
             SSR=0.567500

In [9]:
print(a)
predictor = 1
value = 6.2
print(a[np.argwhere(a[:,predictor] <= value).flatten()])
print(a[np.argwhere(a[:,predictor] > value).flatten()])

[[1.   6.5  2.  ]
 [0.   6.35 2.18]
 [0.   6.2  2.19]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]]
[[0.   6.2  2.19]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]]
[[1.   6.5  2.  ]
 [0.   6.35 2.18]]


In [10]:
from random_forest.tree import Tree

In [11]:
myTree = Tree(a,4)

Branch is created training_data: [[1.   6.5  2.  ]
 [0.   6.35 2.18]
 [0.   6.2  2.19]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]]
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
Branch is created training_data: [[0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]]
WYJEBAŁEM SIĘ NA RYJA KURWA
WYJEBAŁEM SIĘ NA RYJA KURWA
WY

In [12]:
a.shape[0]

12

In [26]:
b = a[a[:,0].argsort()]
print(b)
print(np.unique(b[:,0], return_index=True)[1])

[[0.   6.35 2.18]
 [0.   6.2  2.19]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]
 [1.   6.5  2.  ]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]]
[0 9]


In [31]:
def SSR(a): return ((a.mean() - a)**2).sum()
print(a, SSR(a))
a_mean = a[:,-1].mean()
sum = 0
for i in a[:,-1]:
    sum += (a_mean - i)**2
print(sum)

[[1.   6.5  2.  ]
 [0.   6.35 2.18]
 [0.   6.2  2.19]
 [1.   6.05 1.68]
 [1.   5.9  1.5 ]
 [0.   5.74 1.64]
 [0.   5.59 1.62]
 [0.   5.44 1.56]
 [0.   5.3  1.54]
 [0.   5.26 1.82]
 [0.   5.23 1.86]
 [0.   5.21 1.89]] 197.00969722222223
0.635


In [33]:
test_data = np.array([0.0,   5.23, 1.86])
test_data[1]

5.23